In [1]:
from os import system
system('pip show pandas')

Name: pandas
Version: 1.4.4
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /Users/v.syroiezhin/opt/anaconda3/lib/python3.9/site-packages
Requires: numpy, python-dateutil, pytz
Required-by: datashader, holoviews, hvplot, seaborn, statsmodels, xarray


0

#### Delete the file with the reserved name, if it exists, and create a file to merge the contents

In [2]:
xlsx = 'unite.xlsx'

from glob import glob
def capacity()->list: return glob('*.xlsx') 
def check(message:str): print(message, '\n', capacity(), '\n')

check('Before launch it was like this :')

from os import remove
from os.path import isfile
if isfile(xlsx): 
    remove(xlsx)
    check('Deleted the file named unite.xlsx :')
fxls = sorted(capacity())

from pandas import ExcelWriter
ExcelWriter(xlsx).save()
check('Created a file named unite.xlsx :')

Before launch it was like this : 
 ['Book1.xlsx', 'unite.xlsx', 'Book3.xlsx', 'Book2.xlsx'] 

Deleted the file named unite.xlsx : 
 ['Book1.xlsx', 'Book3.xlsx', 'Book2.xlsx'] 

Created a file named unite.xlsx : 
 ['Book1.xlsx', 'unite.xlsx', 'Book3.xlsx', 'Book2.xlsx'] 



### Looping through files and pages of files to save their contents to a new file

In [3]:
from pandas import ExcelFile, read_excel

def supplementation(this:list, file:str, sheet:str):
    writer = ExcelWriter(file, mode="a", engine="openpyxl")
    this.to_excel(writer, sheet_name=sheet, index=False, header=False)
    writer.save()

def unite(fxls:list):
    list_sheet=[]
    for fxl in fxls:
        ef = ExcelFile(fxl)
        print('\n', fxl, ":", end=' ')
        for sheet in ef.sheet_names:
            list_sheet.append(sheet) if sheet not in list_sheet else list_sheet.append(sheet+'_'+fxl.replace('.xlsx',''))
            supplementation(this=read_excel(ef, sheet_name=sheet), file=xlsx, sheet=list_sheet[-1])
            print(list_sheet[-1], end=' ')

unite(fxls)


 Book1.xlsx : FIRST 
 Book2.xlsx : FIRST_Book2 SECOND 
 Book3.xlsx : FIRST_Book3 SECOND_Book3 THIRD 

#### When we create a file, a page with a known name is also created, we can rename this page

In [4]:
from openpyxl import load_workbook

def rename(file:str, sheet:str):
    fxl = load_workbook(file)
    fxl['Sheet1'].title = sheet
    fxl.save(file)

def verify(file:str, message:str='Result:'):
    fxl = load_workbook(file)
    sheets = fxl.sheetnames
    print(message, sheets)

verify(xlsx, 'Before:')
chief = 'chief'
rename(xlsx, chief)
verify(xlsx, 'After:')

Before: ['Sheet1', 'FIRST', 'FIRST_Book2', 'SECOND', 'FIRST_Book3', 'SECOND_Book3', 'THIRD']
After: ['chief', 'FIRST', 'FIRST_Book2', 'SECOND', 'FIRST_Book3', 'SECOND_Book3', 'THIRD']


### Add data to empty page

In [5]:
cell = 'D4'
content = 'HOWDY GUEST'

def change1cell(file:str, namesheet:str, cell:str, content:str):
    fxl = load_workbook(file)
    sheet = fxl[namesheet]
    sheet[cell]= str(content)
    fxl.save(file)
    print(f'The "{content}" content has been added to cell "{cell}" of the "{namesheet}" page of the "{file}" file.')

change1cell(file=xlsx, namesheet=chief, cell=cell, content=content)

The "HOWDY GUEST" content has been added to cell "D4" of the "chief" page of the "unite.xlsx" file.


### Function to overwrite more than one cell

In [6]:
def fill(file, namesheet, content, mode:str='w', index=False, header=False):
    with ExcelWriter(file, mode=mode, engine="openpyxl", if_sheet_exists=(None if mode=='w' else 'overlay')) as writer:  
        content.to_excel(writer, sheet_name=namesheet, index=index, header=header)

### Write two lines of data to a file, disabling the names of rows and columns

In [7]:
from pandas import DataFrame

df = DataFrame(
    [['1A', '1B','1C'], 
     ['2A', '2B','2C'], 
     ['3A', '3B','3C']],
     columns=['A', 'B', 'C'],
       index=['1', '2', '3'] 
    )

fill(file=xlsx, namesheet=chief, content=df, mode='a')

### Read information from a cell

In [8]:
cell = 'G7'

def scan(file:str, namesheet:str, upto:str, start:str='A1') -> DataFrame:
    tabl = []
    alf = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}
    fxl = load_workbook(file, read_only=False, keep_vba= True)
    cells = fxl[namesheet][start:upto]
    columns = alf.keys()
    for cell in cells: tabl.append([ cell[c].value for c in range(len(cells)) ])
    return DataFrame(tabl, columns=list(columns)[0:(1+alf[upto[0]]-alf[start[0]])], index=[list(range(1,2+int(alf[upto[0]])))])

df = scan(file=xlsx, namesheet=chief, upto=cell)
print(f'In cells "A1:{cell}" of the "{chief}" page of the file "{xlsx}" we got the contents of \n{df}')

In cells "A1:G7" of the "chief" page of the file "unite.xlsx" we got the contents of 
      A     B     C            D     E     F     G
1    1A    1B    1C         None  None  None  None
2    2A    2B    2C         None  None  None  None
3    3A    3B    3C         None  None  None  None
4  None  None  None  HOWDY GUEST  None  None  None
5  None  None  None         None  None  None  None
6  None  None  None         None  None  None  None
7  None  None  None         None  None  None  None


In [9]:
fill(file=xlsx, namesheet=chief, content=df, mode='w', index=True, header=True)
print('The drawn table in the previous block of code has been written to a file')

The drawn table in the previous block of code has been written to a file


### To see the result in the file run this...

In [10]:
system(f'open {xlsx}')

0